In [1]:
import sys
sys.path.append("../")

from utility.potential import *
from utility.propagation import *
from tqdm import tqdm

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

In [2]:
def default_propagation(save_prefix: str, j_init: int, omega_init: int | SpinOne, j_tot: int) -> Propagation:
    propagation = Propagation(save_prefix, j_init, omega_init, j_tot)
    propagation.config.coriolis_omega_max = 4 

    return propagation

# Animations

In [7]:
print("j init: 0, omega init: 0, j_tot: 150")
propagation = default_propagation("coriolis_0_0", 0, 0, 150)
propagation.config.animation = AnimationConfig.All
propagation.provider.transform_gamma = 0.
propagation.config.coriolis_omega_max = 49
propagation = propagation.into_split()

propagation.propagate()
propagation.get_losses()
propagation.save_savers()

print("j init: 1, omega init: 0, j_tot: 150")
propagation = default_propagation("coriolis_1_0", 1, 0, 150)
propagation.config.animation = AnimationConfig.All
propagation.provider.transform_gamma = 0.
propagation.config.coriolis_omega_max = 49
propagation = propagation.into_split()

propagation.propagate()
propagation.get_losses()
propagation.save_savers()

print("j init: 1, omega init: 1, j_tot: 150")
propagation = default_propagation("coriolis_1_1", 1, 1, 150)
propagation.config.animation = AnimationConfig.All
propagation.provider.transform_gamma = 0.
propagation.config.coriolis_omega_max = 49
propagation = propagation.into_split()

propagation.propagate()
propagation.get_losses()
propagation.save_savers()

j init: 0, omega init: 0, j_tot: 150
j init: 1, omega init: 0, j_tot: 150
j init: 1, omega init: 1, j_tot: 150


In [3]:
propagation = default_propagation("free_coriolis", 1, 0, 200)
propagation.config.animation = AnimationConfig.All
propagation.provider.transform_gamma = 0.
propagation.provider.transform_potential = 0.
propagation.config.coriolis_omega_max = 1
propagation = propagation.into_split()

propagation.propagate()
propagation.get_losses()
propagation.save_savers()

In [3]:
def compute_superposition(phase_1, phase_minus_1, suffix, omega_max = 4):
    omega_init = SpinOne(1/3, phase_1, 1/3, phase_minus_1)
    propagation = default_propagation(f"coriolis_1_phase_{suffix}", 1, omega_init, 150)
    propagation.config.animation = AnimationConfig.Polar | AnimationConfig.AngProjection
    propagation.provider.transform_gamma = 0.
    propagation.config.coriolis_omega_max = omega_max
    propagation = propagation.into_split()

    propagation.propagate()
    propagation.get_losses()
    propagation.save_savers()

In [4]:
compute_superposition(0, 0, "0_0")
compute_superposition(np.pi, 0, "pi_0")
compute_superposition(0, np.pi, "0_pi")
compute_superposition(np.pi, np.pi, "pi_pi")
compute_superposition(np.pi/2, 0, "hpi_0")
compute_superposition(0, np.pi/2, "0_hpi")
compute_superposition(np.pi/2, np.pi/2, "hpi_hpi")
compute_superposition(np.pi, np.pi/2, "pi_hpi")
compute_superposition(np.pi/2, np.pi, "hpi_pi")

populating coriolis


100%|██████████| 1024/1024 [00:36<00:00, 28.27it/s]


populating coriolis


100%|██████████| 1024/1024 [00:41<00:00, 24.52it/s]


populating coriolis


...

100%|██████████| 1024/1024 [00:36<00:00, 28.07it/s]


populating coriolis


100%|██████████| 1024/1024 [00:37<00:00, 27.59it/s]


populating coriolis


100%|██████████| 1024/1024 [00:38<00:00, 26.27it/s]


In [4]:
for omega_max in [1, 4, 10, 20]:
    compute_superposition(0, 0, f"0_0_omega_max_{omega_max}", omega_max=omega_max)

populating coriolis


100%|██████████| 1024/1024 [00:22<00:00, 45.40it/s]


populating coriolis


100%|██████████| 1024/1024 [00:30<00:00, 33.64it/s]


populating coriolis


100%|██████████| 1024/1024 [01:01<00:00, 16.63it/s]


populating coriolis


100%|██████████| 1024/1024 [04:35<00:00,  3.72it/s]


# Single losses

In [3]:
print("j init: 0, omega init: 0, j_tot: 0")
propagation = default_propagation("coriolis", 0, 0, 0)
propagation = propagation.into_split()
propagation.propagate()
print(propagation.get_losses())

print("j init: 1, omega init: 0, j_tot: 0")
propagation = default_propagation("coriolis", 1, 0, 0)
propagation = propagation.into_split()
propagation.propagate()
print(propagation.get_losses())

print("j init: 1, omega init: 1, j_tot: 1")
propagation = default_propagation("coriolis", 1, 1, 1)
propagation = propagation.into_split()
propagation.propagate()
print(propagation.get_losses())

j init: 0, omega init: 0, j_tot: 0
[0.18949809258435846, 0.07631859937973784]
j init: 1, omega init: 0, j_tot: 1
[0.24269727220961246, 0.07029796121983534]
j init: 1, omega init: 1, j_tot: 1
[0.16289946681036382, 0.07933036011344008]


# Cross sections

In [4]:
def calc_losses(j_init: int, omega_init: int | SpinOne, omega_max: int):
    losses = CumulativeLosses(j_init)

    for j_tot in tqdm(losses.j_totals):
        propagation = default_propagation("coriolis", j_init, omega_init, j_tot)
        propagation.config.coriolis_omega_max = omega_max
        propagation = propagation.into_split()

        propagation.propagate()
        losses.extract_loss(propagation)
        
    losses.save_losses(f"coriolis_losses_omega_max_{omega_max}_{j_init}_{omega_init}")

In [5]:
calc_losses(0, 0, 1)
calc_losses(1, 0, 1)
calc_losses(1, 1, 1)
calc_losses(1, SpinOne(1/3, 0, 1/3, 0), 1)

100%|██████████| 50/50 [08:39<00:00, 10.40s/it]


In [6]:
calc_losses(0, 0, 2)
calc_losses(1, 0, 2)
calc_losses(1, 1, 2)
calc_losses(1, SpinOne(1/3, 0, 1/3, 0), 2)

100%|██████████| 50/50 [09:48<00:00, 11.78s/it]


In [7]:
calc_losses(0, 0, 4)
calc_losses(1, 0, 4)
calc_losses(1, 1, 4)
calc_losses(1, SpinOne(1/3, 0, 1/3, 0), 4)

100%|██████████| 50/50 [11:49<00:00, 14.18s/it]


In [8]:
calc_losses(0, 0, 8)
calc_losses(1, 0, 8)
calc_losses(1, 1, 8)
calc_losses(1, SpinOne(1/3, 0, 1/3, 0), 8)

100%|██████████| 50/50 [14:57<00:00, 17.95s/it]
